In [1]:
import os
import sys

_PATH = '/Users/winston/mygitrep/sys23/'  # TODO
os.chdir(_PATH)
sys.path.append(_PATH)

In [2]:
import warnings
warnings.simplefilter("ignore")
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn
# seaborn.set_style("darkgrid")
seaborn.set_style("whitegrid")
plt.rc("figure", figsize=(16, 6))
# plt.rc("figure", figsize=(6, 3))
plt.rc("savefig", dpi=90)
# plt.rc("font", family="sans-serif")
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
plt.rc("font", size=12)
# plt.rc("font", size=10)
plt.rcParams["date.autoformatter.hour"] = "%H:%M:%S"

In [3]:
from api.load_tushare import conf_init, next_calendar_date
import pandas as pd
import numpy as np

config_path = f'{_PATH}/config_stk.yaml'
conf = conf_init(conf_path=config_path)

# begin_date = '2015-01-01'
begin_date = '2018-01-01'
end_date = '2022-12-31'
bd_1 = next_calendar_date(begin_date, delta=-5, lfmt='%Y-%m-%d', rfmt='%Y-%m-%d')

In [4]:
conf.keys()

dict_keys(['status', 'tushare_token', 'api_path', 'path', 'year_length', 'csi_pool'])

## 各指数内统计

In [5]:
conf['csi_pool']

{'CSI300': '399300.SZ',
 'CSI500': '000905.SH',
 'CSI800': '000906.SH',
 'CSI1000': '000852.SH'}

In [6]:
close_raw = pd.read_csv(conf['path']['close'], index_col=0, parse_dates=True).loc[begin_date:end_date]

tmp = []
for kind_pool in conf['csi_pool'].keys():

    stk_pool = pd.read_csv(
        conf['path'][kind_pool], index_col=0, parse_dates=True
    ).loc[begin_date:end_date]

    stk_pool_by_date = stk_pool.apply(lambda x: x[x>0].index.to_list(), axis=1)

    df = close_raw.apply(
        lambda s: s.loc[set(s.index).intersection(set(stk_pool_by_date.loc[s.name]))].apply(
            lambda x: int(x*100%10) if not np.isnan(x) else 99
        ).value_counts(),
        axis=1
    )
    df1 = df.sum().rename((kind_pool, '出现次数'))
    df1.rename(index={99: '缺失'}, inplace=True)
    df2 = (df1 / df1.sum()).rename((kind_pool, '频率'))
    tmp.append(df1)
    tmp.append(df2)
del df1, df2
df = pd.concat(tmp, axis=1)
del tmp
df.loc['总计'] = df.sum()
df.index.name = 'DP'
df_repr = df.applymap(
    lambda x: str(int(x)) if x > 2 else f'{x * 100:2.1f}%')
df.to_excel(f"{conf['path']['demo']}lucky_number/样本数量及其分布_ALL.xlsx")
df_repr.to_csv(f"{conf['path']['demo']}lucky_number/样本数量及其分布_ALL.csv")
print(df_repr)

table01 = df.copy()
del df, df_repr, close_raw

    CSI300           CSI500           CSI800          CSI1000        
      出现次数      频率     出现次数      频率     出现次数      频率     出现次数      频率
DP                                                                   
0   110336   15.1%   154912   12.7%   265248   13.6%   322284   13.3%
1    67056    9.2%   114158    9.4%   181214    9.3%   225170    9.3%
2    65884    9.0%   112724    9.3%   178608    9.2%   223550    9.2%
3    64392    8.8%   113622    9.4%   178014    9.2%   224836    9.3%
4    62588    8.6%   110704    9.1%   173292    8.9%   216682    8.9%
5    77042   10.6%   126268   10.4%   203310   10.5%   252540   10.4%
6    64510    8.8%   112666    9.3%   177176    9.1%   224664    9.2%
7    63986    8.8%   110744    9.1%   174730    9.0%   219350    9.0%
8    73568   10.1%   123376   10.2%   196944   10.1%   249162   10.3%
9    73518   10.1%   122210   10.1%   195728   10.1%   243114   10.0%
缺失    6120    0.8%    13616    1.1%    19736    1.0%    28648    1.2%
总计  729000  100.0%  

## CSI500

In [7]:
kind_pool = 'CSI500'

stk_pool = pd.read_csv(
    conf['path'][kind_pool], index_col=0, parse_dates=True
).loc[bd_1:end_date]


stk_pool_by_date = stk_pool.apply(lambda x: x[x>0].index.to_list(), axis=1)
del stk_pool

In [8]:
close_raw = pd.read_csv(conf['path']['close'], index_col=0, parse_dates=True).loc[begin_date:end_date]

df = close_raw.apply(
    lambda s: s.loc[s.index.intersection(stk_pool_by_date.loc[s.name])].apply(
        lambda x: int(x*100%10) if not np.isnan(x) else 99
    ).value_counts(),
    axis=1
)
del close_raw
# df.isna().sum(axis=1).cumsum().plot(title=f'{kind_pool}日股价缺失数量累积')
# plt.show()

grouped = df.groupby(df.index.to_series().apply(lambda x: x.year))
appr_cnt = grouped.apply(sum)
appr_freq = appr_cnt.apply(lambda s: s / s.sum(), axis=1)
appr_cnt_freq = pd.concat(
    [appr_cnt.stack().rename('出现次数'),
     appr_freq.stack().rename('频率')],
    axis=1
).reset_index()
appr_cnt_freq.columns = ['年份', 'DP', '出现次数', '频率']
appr_cnt_freq = appr_cnt_freq.pivot(index='DP', columns=['年份'], values=['出现次数', '频率'])

appr_cnt_freq = appr_cnt_freq.reset_index()
cols = [('DP', '')]
for year in range(int(begin_date[:4]), int(end_date[:4])+1):
    cols += [('出现次数', year), ('频率', year)]

appr_cnt_freq = appr_cnt_freq[cols].set_index('DP')
appr_cnt_freq.columns.names = [None, None]
appr_cnt_freq.rename(index={99: '缺失'}, inplace=True)
appr_cnt_freq.loc['总计'] = appr_cnt_freq.sum()

appr_cnt_freq_repr = appr_cnt_freq.applymap(
    lambda x: str(int(x)) if x > 1 else f'{x * 100:2.1f}%')
appr_cnt_freq.to_excel(f"{conf['path']['demo']}lucky_number/样本数量及其分布_{kind_pool}.xlsx")
appr_cnt_freq_repr.to_csv(f"{conf['path']['demo']}lucky_number/样本数量及其分布_{kind_pool}.csv")

table02 = appr_cnt_freq.copy()
del df, grouped, appr_cnt, appr_freq, appr_cnt_freq, appr_cnt_freq_repr
# appr_cnt_freq_repr

In [9]:
close_adj: pd.DataFrame = pd.read_csv(
    conf['path']['closeAdj'], index_col=0, parse_dates=True).loc[bd_1: end_date]
open_adj: pd.DataFrame = pd.read_csv(
    conf['path']['openAdj'], index_col=0, parse_dates=True).loc[bd_1: end_date]
rtn_c2c = close_adj.pct_change().loc[begin_date:]  # T-1 close to T0 close
rtn_c2c_lag1 = close_adj.pct_change().shift(1).loc[begin_date:]  # T-2 close to T-1 close
rtn_c2o = (open_adj / close_adj.shift(1) - 1).loc[begin_date:]  # T-1 close to T0 open
del close_adj, open_adj

# 直接去除涨跌停
rtn_c2o[rtn_c2o > 0.101] = np.nan
rtn_c2o[rtn_c2o < -0.101] = np.nan
rtn_c2c_lag1[rtn_c2c_lag1 > 0.101] = np.nan
rtn_c2c_lag1[rtn_c2c_lag1 < -0.101] = np.nan
rtn_c2c[rtn_c2c > 0.101] = np.nan
rtn_c2c[rtn_c2c < -0.101] = np.nan

# 超额收益率
rtn_c2o = rtn_c2o.apply(lambda s : s - s.mean(), axis=1)
rtn_c2c = rtn_c2c.apply(lambda s : s - s.mean(), axis=1)
rtn_c2c_lag1 = rtn_c2c_lag1.apply(lambda s : s - s.mean(), axis=1)

df1 = rtn_c2o.apply(lambda s: s.loc[s.index.intersection(
    stk_pool_by_date.loc[s.name])],axis=1).stack().reset_index()
df1.columns = ['trade_date', 'stock_code', 'AOC']
df2 = rtn_c2c.apply(lambda s: s.loc[s.index.intersection(
    stk_pool_by_date.loc[s.name])],axis=1).stack().reset_index()
df2.columns = ['trade_date', 'stock_code', 'ACC']
# plt.plot(df1.groupby('trade_date').count().iloc[:, 0], label='c2o')
# plt.plot(df2.groupby('trade_date').count().iloc[:, 0], label='c2c')
# plt.legend()
# plt.title('CSI500收益率得不到');
panel: pd.DataFrame = df1.merge(df2, on=['trade_date', 'stock_code'], how='outer')
del df1, df2, rtn_c2c, rtn_c2o

df = rtn_c2c_lag1.apply(lambda s: s.loc[s.index.intersection(
    stk_pool_by_date.loc[s.name])], axis=1).stack().reset_index()
df.columns = ['trade_date', 'stock_code', 'ACC_L1']
panel = panel.merge(df, on=['trade_date', 'stock_code'], how='left')
del df, rtn_c2c_lag1

close_raw0 = pd.read_csv(conf['path']['close'], index_col=0, parse_dates=True).loc[bd_1:end_date]
tail_number = close_raw0.apply(
    lambda s: s.loc[s.index.intersection(stk_pool_by_date.loc[s.name])].apply(
        lambda x: int(x*100%10) if not np.isnan(x) else -1),
    axis=1
).shift(1).loc[begin_date:]  # 昨日收盘的尾数
tail_number2 = close_raw0.apply(
    lambda s: s.loc[s.index.intersection(stk_pool_by_date.loc[s.name])].apply(
        lambda x: int(x*100%100) // 5 * 5 if not np.isnan(x) else -1),
    axis=1
).shift(1).loc[begin_date:]  # 昨日收盘的尾数2
df01 = tail_number.stack().reset_index()
df01.columns = ['trade_date', 'stock_code', 'DP']
df02 = tail_number2.stack().reset_index()
df02.columns = ['trade_date', 'stock_code', 'DP2']
del tail_number, tail_number2

panel = panel.merge(df01, on=['trade_date', 'stock_code'], how='left')
panel = panel.merge(df02, on=['trade_date', 'stock_code'], how='left')
del df01, df02


df = close_raw0.applymap(lambda x: int(x) if not np.isnan(x) else x)
sgn1 = (df > df.shift(1)).loc[begin_date:]  # 当日收盘相比昨日收盘，向上跨越了整数位
sgn2 = (df < df.shift(1)).loc[begin_date:]  # 当日收盘相比昨日收盘，向下跨越了整数位
sgn3 = (df.shift(-1) > df).loc[begin_date:]  # 明日收盘相比今日收盘，向上跨越了整数位
sgn4 = (df.shift(-1) < df).loc[begin_date:]  # 明日收盘相比今日收盘，向下跨越了整数位
del df

for df, kw in zip([sgn1, sgn2, sgn3, sgn4], ['AP', 'AD', 'BP', 'BD']):
    df = df.apply(
        lambda s: s.loc[s.index.intersection(stk_pool_by_date.loc[s.name])], axis=1
    ).stack().reset_index()
    df.columns = ['trade_date', 'stock_code', kw]
    panel = panel.merge(df, on=['trade_date', 'stock_code'], how='left')
    
del sgn1, sgn2, sgn3, sgn4
del close_raw0

In [10]:
df = panel.groupby('DP').agg(
    观测数 = ('DP', 'count'),
    C2C_均值 = ('ACC', 'mean'),
    C2C_中位数 = ('ACC', 'median'),
    C2C_t值 = ('ACC', lambda x: x.mean()/x.std()*np.sqrt(x.count())),
    C2O_均值 = ('AOC', 'mean'),
    C2O_中位数 = ('AOC', 'median'),
    C2O_t值 = ('AOC', lambda x: x.mean()/x.std()*np.sqrt(x.count())),
)

df = df.loc[df.index>=0]
df.index = df.index.to_series().astype(int)

df_repr = df.applymap(lambda x: f'{x*100:2.3f}%' if abs(x) < 0.01 else round(x, 2) if isinstance(x, float) else x)
# print(df_repr.to_markdown())
print(df_repr)

df_repr.to_csv(conf['path']['demo']+'价格尾数（分位）与随后的股票收益率.csv')
df.to_excel(conf['path']['demo']+'价格尾数（分位）与随后的股票收益率.xlsx')
table03 = df.copy()
del df, df_repr

        观测数   C2C_均值  C2C_中位数  C2C_t值  C2O_均值  C2O_中位数  C2O_t值
DP                                                            
0   4952320  -0.009%  -0.205%   -8.23  0.029%   0.008%   62.70
1   3649792   0.023%  -0.182%   19.55  0.039%   0.009%   73.27
2   3604352   0.026%  -0.198%   21.61  0.033%   0.005%   62.02
3   3631104   0.002%  -0.207%    1.70  0.039%   0.005%   71.26
4   3538304  -0.001%  -0.195%   -0.99  0.019%  -0.002%   35.66
5   4036992  -0.002%  -0.189%   -1.42  0.022%   0.002%   44.08
6   3600256  -0.002%  -0.195%   -1.81  0.029%   0.009%   55.86
7   3540032   0.005%  -0.189%    4.47  0.030%   0.005%   55.42
8   3944448  -0.005%  -0.195%   -4.67  0.018%   0.002%   35.07
9   3908224   0.001%  -0.196%    1.08  0.029%   0.007%   56.18


In [11]:
df = panel.groupby('DP2').agg(
    观测数 = ('DP2', 'count'),
    C2C_均值 = ('ACC', 'mean'),
    C2C_中位数 = ('ACC', 'median'),
    C2C_t值 = ('ACC', lambda x: x.mean()/x.std()*np.sqrt(x.count())),
    C2O_均值 = ('AOC', 'mean'),
    C2O_中位数 = ('AOC', 'median'),
    C2O_t值 = ('AOC', lambda x: x.mean()/x.std()*np.sqrt(x.count())),
)

df = df.loc[df.index>=0]
df.index = df.index.to_series().apply(lambda x: f'{int(x):02d}~{int(x)+4:02d}')

df_repr = df.applymap(lambda x: f'{x*100:2.3f}%' if abs(x) < 0.01 else round(x, 2) if isinstance(x, float) else x)
# print(df_repr.to_markdown())
print(df_repr)

df_repr.to_csv(conf['path']['demo']+'价格尾数（毛、分位）与随后的股票收益率.csv')
df.to_excel(conf['path']['demo']+'价格尾数（毛、分位）与随后的股票收益率.xlsx')
table04 = df.copy()
del df, df_repr

           观测数   C2C_均值  C2C_中位数  C2C_t值  C2O_均值  C2O_中位数  C2O_t值
DP2                                                              
00~04  2154304   0.005%  -0.199%    3.07  0.050%   0.018%   70.43
05~09  2015552   0.001%  -0.184%    0.68  0.038%   0.016%   55.04
10~14  1954496   0.004%  -0.198%    2.70  0.045%   0.012%   62.55
15~19  2062144  -0.007%  -0.184%   -4.53  0.036%   0.020%   54.16
20~24  1884800   0.017%  -0.201%   10.23  0.035%   0.013%   47.34
25~29  1960832  -0.023%  -0.201%  -14.20  0.014%  -0.002%   19.44
30~34  1884288   0.031%  -0.190%   18.51  0.030%   0.004%   39.75
35~39  1889856   0.011%  -0.189%    6.41  0.018%   0.001%   24.01
40~44  1855360   0.017%  -0.209%    9.87  0.038%   0.007%   50.96
45~49  1764544   0.026%  -0.182%   15.08  0.025%   0.004%   31.69
50~54  1932864   0.007%  -0.199%    4.44  0.028%   0.005%   36.42
55~59  1798144  -0.019%  -0.215%  -10.90  0.023%  -0.000%   29.22
60~64  1856192  -0.006%  -0.195%   -3.44  0.029%   0.005%   38.25
65~69  189

## 回归

In [12]:
panel = panel[['trade_date', 'stock_code', 'ACC', 'ACC_L1', 'AP', 'AD', 'BP', 'BD']].dropna()

In [13]:
import statsmodels.formula.api as smf

In [14]:
res = panel.groupby('stock_code').apply(
    lambda s: smf.ols(formula='ACC ~ ACC_L1 + AP + AD + BP + BD', data=s.dropna()).fit().params[1:]
)

In [15]:
df = res.apply(['count', 'mean', lambda x: x.mean()/x.std()*np.sqrt(x.count())]).T
df.columns = ['N', '均值', 't值']
df['N'] = df['N'].astype(int)
df.index = ['AP', 'AD', 
            'BP', 'BD', 
            'R_{t-1}']
df.index.name = '对应系数'
df['备注'] = ['时间t价格向上跨越整数价位', '时间t价格向下跨越整数价位', 
             '时间t+1价格向上跨越整数价位', '时间t+1价格向下跨越整数价位', 
             '时间t-1期的收益率']
df_repr = df.round({'N': 0, '均值': 3, 't值': 2})

df.to_excel(conf['path']['demo'] + '回归系数.xlsx')
df_repr.to_csv(conf['path']['demo'] + '回归系数.csv')
table05 = df.copy()

In [16]:
print(df_repr)

           N     均值     t值               备注
对应系数                                       
AP       937  0.029  65.57    时间t价格向上跨越整数价位
AD       937 -0.021 -65.44    时间t价格向下跨越整数价位
BP       937  0.006  26.09  时间t+1价格向上跨越整数价位
BD       937 -0.004 -24.31  时间t+1价格向下跨越整数价位
R_{t-1}  937 -0.004  -1.89       时间t-1期的收益率
